In [ ]:
import re
from copy import deepcopy


In [ ]:
ifmt = dict[int, list[int|list[int]]]

def read(file: str) -> ifmt:
    data = {}
    with open(file) as f:
        for line in f:
            p = line.strip().split()            
            loc = p[1]
            flow = int(p[4].split("=")[-1].strip(";"))
            neighbors = [x.strip(",") for x in p[9:]]

            data[loc] = [flow, neighbors]
    return data


example = read("inputs/day-16-example.txt")
inputs = read("inputs/day-16.txt")


## Part 1

Use beam search for 1 person and 30 steps.
Iterate to find a large enough beam.

In [ ]:
def part1(input_data, beamsize: int = 1000):
    t = 30
    beams = [[0, [], ["AA"], input_data]]
    while t > 0:
        new_beams = []
        # Take all actions for each beam, keep the top N
        for value, open, steps, data in beams:
            cur = steps[0]
            # Open
            if data[cur][0] > 0:
                nd = deepcopy(data)
                nd[cur][0] = 0
                new_beams += [
                    [
                        value + (t - 1) * data[cur][0],
                        [(cur, 30 - t + 1)] + open,
                        [cur] + steps,
                        nd,
                    ]
                ]
            # Neighbors
            for nn in data[cur][-1]:
                new_beams += [[value, open, [nn] + steps, data]]
        v0 = max(b[0] for b in beams)
        beams = sorted(new_beams, key=lambda x: x[0], reverse=True)[:beamsize]
        v1 = max(b[0] for b in beams)
        # print(v1, v0, v1 - v0)
        t -= 1
    return beams


In [ ]:
part1(example, 1000)[0][0]


In [ ]:
part1(inputs, 1000)[0][0]


## Part 2

Cooperate with elephants!

In [ ]:
ofmt = list[int | list[int] | ifmt]


def part2(input_data: ifmt, beamsize: int = 1000) -> list[ofmt]:
    t = 26
    beams = [[0, [[], []], [["AA"], ["AA"]], input_data]]
    while t > 0:
        new_beams = []
        # Take all actions for each beam, keep the top N
        for value, (oa, ob), (sa, sb), data in beams:
            ca, cb = sa[0], sb[0]

            # Both open
            if data[ca][0] > 0 or data[cb][0] > 0:
                odata = deepcopy(data)
                v = 0
                if odata[ca][0] > 0:
                    v += odata[ca][0] * (t - 1)
                    odata[ca][0] = 0
                    oa = [ca] + oa
                if odata[cb][0] > 0:
                    v += odata[cb][0] * (t - 1)
                    odata[cb][0] = 0
                    ob = [cb] + ob
                new_beams += [[value + v, [oa, ob], [[ca] + sa, [cb] + sb], odata]]

            # One and one
            if data[ca][0] > 0:
                odata = deepcopy(data)
                v = odata[ca][0] * (t - 1)
                odata[ca][0] = 0
                oa = [ca] + oa
                for nb in data[cb][-1]:
                    new_beams += [[value + v, [oa, ob], [[ca] + sa, [nb] + sb], odata]]

            if data[cb][0] > 0:
                odata = deepcopy(data)
                v = odata[cb][0] * (t - 1)
                odata[cb][0] = 0
                oa = [cb] + ob
                for na in data[ca][-1]:
                    new_beams += [[value + v, [oa, ob], [[na] + sa, [cb] + sb], odata]]

            # Both move
            for na in data[ca][-1]:
                for nb in data[cb][-1]:
                    new_beams += [[value, [oa, ob], [[na] + sa, [nb] + sb], data]]

        beams = sorted(new_beams, key=lambda x: x[0], reverse=True)[:beamsize]
        t -= 1

    return beams


In [ ]:
part2(example, beamsize=5000)[0][0]

In [ ]:
part2(inputs, beamsize=5000)[0][0]